## Welcome to ACME eCommerce LLC 

This is your first day on the job. Everyone of our empoyees is its own special flower...

Now make us money

In [2]:

ACME eCommerce is in the business of selling things to people. Your first task is to tell me how many people we had on the site.

First lets setup some visit data first. Please open the notebook [03 - Producer](/notebooks/03%20-%20Producer.ipynb) in a new tab and run through all the code there. Then return here.

SyntaxError: invalid syntax (<ipython-input-2-31ab4e299e67>, line 2)

In [3]:
import pykafka
 
docker_ip = '192.168.99.100'
client = pykafka.KafkaClient(hosts="{}:9092".format(docker_ip))

In [4]:
topic = client.topics['visit_log1'] 

In [6]:
c = topic.get_simple_consumer()

In [16]:
topic.partitions

{0: <pykafka.partition.Partition at 0x1077d7d10 (id=0)>}

In [8]:
consumer = topic.get_simple_consumer()

In [12]:
message = consumer.consume(block=False)

In [13]:
message.offset

1

In [14]:
# WE HAVE OUR FIRST MESSAGE.
message.value

'1441920363.95 [main] INFO Index.html user:2129'

We can loop through all the messages with this code. But be careful if you have lots of messages in your topic!

In [15]:
%%time
count = 0

while True:
    message = consumer.consume(block=False)
    if not message:
        break
    # YOUR CODE HERE
    count += 1
print "We counsumed: {} messages".format(count)

We counsumed: 498 messages
CPU times: user 3.86 ms, sys: 1.46 ms, total: 5.32 ms
Wall time: 4.06 ms


So your first coding task. 
Given this stream answers your bosses first question. How many unique users have visited the site?

In [65]:
consumer = topic.get_simple_consumer()

In [66]:
%%time

while True:
    message = consumer.consume(block=False)
    if not message:
        break
    # YOUR CODE HERE

CPU times: user 17 µs, sys: 4 µs, total: 21 µs
Wall time: 21.9 µs


# Stateful vs Stateless Processing